<a href="https://colab.research.google.com/github/Vengeance99/Learning-Representation-of-data-towards-Categorisation/blob/main/IDEC_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
x=np.empty((512,512))

In [ ]:
for i in range(len(x)):
  for j in range(len(x)):
    if i==j:
       x[i][j]=1

In [ ]:
import time
import copy
import pickle
# from barbar import Bar
import numpy as np 
import pandas as pd 
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from torchvision.datasets import CIFAR10
from torchvision.datasets import MNIST
import torchvision.datasets as datasets
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
# from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
# %matplotlib inline
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

import torch
import torchvision

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchsummary import summary
import torchvision.datasets as datasets
from tqdm import tqdm
from pathlib import Path
import gc
RANDOMSTATE = 0
import os

In [ ]:
print(x)

In [ ]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from sklearn.cluster import MiniBatchKMeans, KMeans


trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

train_set = dset.FashionMNIST(root='/content/drive/MyDrive', train=True, transform=trans, download=True)
test_set = dset.FashionMNIST(root='/content/drive/MyDrive', train=False, transform=trans, download=True)
batch_size = 100
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

In [ ]:
print(len(train_loader))

In [ ]:
import time
timingResult = {}
def logTime(theName, currentTime):
    if theName not in timingResult:
        timingResult[theName] = time.time() - currentTime
    else:
        timingResult[theName] = timingResult[theName] + (time.time() - currentTime)
    currentTime = time.time()
    return currentTime

def printTiming(name):
    print('======== timing for {}: {} ======='.format(name,timingResult[name]))

In [ ]:
class DEC_AE(nn.Module):
    def __init__(self, num_classes, num_features):
        super(DEC_AE,self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(28*28,500)
        self.fc2 = nn.Linear(500,500)
        self.fc3 = nn.Linear(500,2000)
        self.fc4 = nn.Linear(2000,num_features)
        self.relu = nn.ReLU()
        self.fc_d1 = nn.Linear(500,28*28)
        self.fc_d2 = nn.Linear(500,500)
        self.fc_d3 = nn.Linear(2000,500)
        self.fc_d4 = nn.Linear(num_features,2000)
        self.alpha = 1.0
        # self.clusterCenter = nn.Parameter(torch.zeros(num_classes,num_features))
        # self.pretrainMode = True
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform(m.weight)

    # def setPretrain(self,mode):
    #     """To set training mode to pretrain or not, 
    #     so that it can control to run only the Encoder or Encoder+Decoder"""
    #     self.pretrainMode = mode
    # def updateClusterCenter(self, cc):
    #     """
    #     To update the cluster center. This is a method for pre-train phase.
    #     When a center is being provided by kmeans, we need to update it so
    #     that it is available for further training
    #     :param cc: the cluster centers to update, size of num_classes x num_features
    #     """
    #     self.clusterCenter.data = torch.from_numpy(cc)
    # def getTDistribution(self,x, clusterCenter):
    #     """
    #     student t-distribution, as same as used in t-SNE algorithm.
    #      q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
         
    #      :param x: input data, in this context it is encoder output
    #      :param clusterCenter: the cluster center from kmeans
    #      """
    #     xe = torch.unsqueeze(x,1).cuda() - clusterCenter.cuda()
    #     q = 1.0 / (1.0 + (torch.sum(torch.mul(xe,xe), 2) / self.alpha))
    #     q = q ** (self.alpha + 1.0) / 2.0
    #     q = (q.t() / torch.sum(q, 1)).t() #due to divison, we need to transpose q
    #     return q
        
    def forward(self,x):
        x = x.view(-1, 1*28*28)
        # 32x32x1
        x = self.dropout(x)
        # 32x32x1
        x = self.fc1(x)
        # 17x17x50
        x = self.relu(x)
        # 17x17x50
        x = self.fc2(x)
        # 17x17x50
        x = self.relu(x)
        # 9x9x50
        x = self.fc3(x)
        # 17x17x50
        x = self.relu(x)
        x = self.fc4(x)
        # 9x9x50
        x_ae = x
        #if not in pretrain mode, we only need encoder
        # if self.pretrainMode == False:
        #     return x, self.getTDistribution(x,self.clusterCenter)
        # 1x68
        ##### encoder is done, followed by decoder #####
        # 1x68
        x = self.fc_d4(x)
        # 1x4050
        x = self.relu(x)
        # 1x4050
        x = self.fc_d3(x)
        # 1x4050
        x = self.relu(x)
        x = self.fc_d2(x)
        # 1x4050
        x = self.relu(x)
        x = self.fc_d1(x)
        x_de = x.view(-1,1,28,28)
        # 1x4050
        return x_ae, x_de

In [ ]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from scipy.optimize import linear_sum_assignment as linear_assignment
nmi = normalized_mutual_info_score
ari = adjusted_rand_score


def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    # from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    # for i,j in ind:
    ind=list(ind)
    print(ind[0])
    # print(i for )
    return (sum([w[i, j] for i, j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size)

In [ ]:
# class DEC:
#     """The class for controlling the training process of DEC"""
#     def __init__(self,n_clusters,alpha=1.0):
#         self.n_clusters=n_clusters
#         self.alpha = alpha
        

#     def target_distribution(q):
#         weight = q ** 2 / q.sum(0)
#         #print('q',q)
#         return Variable((weight.t() / weight.sum(1)).t().data, requires_grad=True)
#     def logAccuracy(self,pred,label):
#         print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
#           % (acc(label, pred), nmi(label, pred)))

#     def kld(q,p):
#         res = torch.sum(p*torch.log(p/q),dim=-1)
#         return res
    
def validateOnCompleteTestData(test_loader,model):
    model.eval()
    lat=[]
    lab=[]
    for i,(data,label )in enumerate(test_loader):
            data=data.to(device)
            label=label.to(device)
            x_ae,_ = model(data)
            lat.extend(x_ae.data.cpu().numpy())
            lab.extend(label.cpu().numpy())
    to_eval=np.array(lat)
    true_labels=np.array(lab)

            # to_eval=np.array()
    # to_eval = np.array([model(d[0].cuda())[0].data.cpu().numpy() for i,d in enumerate(test_loader)])
    # true_labels = np.array([d[1].cpu().numpy() for i,d in enumerate(test_loader)])
    print(to_eval.shape)
    print(true_labels.shape)
    # to_eval = np.reshape(to_eval,(to_eval.shape[0]*to_eval.shape[1],to_eval.shape[2]))
    # true_labels = np.reshape(true_labels,true_labels.shape[0]*true_labels.shape[1])
    km = KMeans(n_clusters=len(np.unique(true_labels)), n_init=20)
    y_pred = km.fit_predict(to_eval)
    print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                  % (acc(true_labels, y_pred), nmi(true_labels, y_pred)))
    currentAcc = acc(true_labels, y_pred)
    return currentAcc
    
def pretrain(train_loader, test_loader, epochs):
    dec_ae = DEC_AE(10,10).cuda() #auto encoder
    mseloss = nn.MSELoss()
    optimizer = optim.SGD(dec_ae.parameters(),lr = 1, momentum=0.9)
    best_acc = 0.0
    for epoch in range(epochs):
        dec_ae.train()
        running_loss=0.0
        for i,(data,label )in enumerate(train_loader):
            # print(type(data))
            data=data.to(device)
            label=label.to(device)
            # x, label = data
            # x, label = Variable(x).cuda(),Variable(label).cuda()
            optimizer.zero_grad()
            x_ae,x_de = dec_ae(data)
            loss = F.mse_loss(x_de,data,reduce=True) 
            loss.backward()
            optimizer.step()
            # x_eval = x.data.cpu().numpy()
            # label_eval = label.data.cpu().numpy()
            # running_loss += loss.data.cpu().numpy()
            running_loss += loss.item() * data.size(0)
            if i % 100 == 99:    # print every 100 mini-batches
                print('[%d, %5d] loss: %.7f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0
        #now we evaluate the accuracy with AE
        dec_ae.eval()
        currentAcc =validateOnCompleteTestData(test_loader,dec_ae)
        if currentAcc > best_acc:                
            torch.save(dec_ae,'/content/drive/MyDrive/my_models/bestModel'.format(best_acc))
            best_acc = currentAcc
    #     # print(f"loss for epoch{i}:")        
    # def clustering(self,mbk,x,model):
    #     model.eval()
    #     y_pred_ae,_ = model(x)
    #     y_pred_ae = y_pred_ae.data.cpu().numpy()
    #     y_pred = mbk.partial_fit(y_pred_ae) #seems we can only get a centre from batch
    #     self.cluster_centers = mbk.cluster_centers_ #keep the cluster centers
    #     model.updateClusterCenter(self.cluster_centers)
    # def train(self,train_loader, test_loader, epochs):
    #     """This method will start training for DEC cluster"""
    #     ct = time.time()
    #     dec_ae = DEC_AE(10,10).cuda() 
    #     best_acc = 0.0
    #     model = torch.load("/content/drive/MyDrive/my_models/bestModel").cuda()
    #     model.setPretrain(False)
    #     optimizer = optim.SGD([\
    #          {'params': model.parameters()}, \
    #         ],lr = 0.01, momentum=0.9)
    #     print('Initializing cluster center with pre-trained weights')
    #     mbk = MiniBatchKMeans(n_clusters=self.n_clusters, n_init=20, batch_size=batch_size)
    #     got_cluster_center = False
    #     for epoch in range(epochs):
    #         for i,(data,label) in enumerate(train_loader):
    #             # x, label = data
    #             # x = Variable(x).cuda()
    #             data=data.to(device)
    #             label=label.to(device)
    #             optimizer.zero_grad()
    #             # data=np.array(data)
    #             # data=torch.from_numpy(data)
    #             # data=data.to(device)

    #             #step 1 - get cluster center from batch
    #             #here we are using minibatch kmeans to be able to cope with larger dataset.
    #             if not got_cluster_center:
    #                 self.clustering(mbk,data,model)
    #                 if epoch > 1:
    #                     got_cluster_center = True
    #             else:
    #                 model.train()
    #                 #now we start training with acquired cluster center
    #                 feature_pred,q = model(data)

    #                 #get target distribution
    #                 p = self.target_distribution(q)
    #                 #print('q',q,'p',p)
    #                 loss1 = self.kld(q,p).mean()
    #                 x_ae,x_de = dec_ae(data)
    #                 loss2 = F.mse_loss(x_de,data,reduce=True) 
    #                 loss=loss1+loss2
    #                 loss.backward()
    #                 optimizer.step()
    #         currentAcc = self.validateOnCompleteTestData(test_loader,model)
    #         if currentAcc > best_acc:                
    #             torch.save(model,'/content/drive/MyDrive/my_models/DECbestModel'.format(best_acc))
    #             best_acc = currentAcc

In [ ]:
#now start training
# import random
# random.seed(7)
# dec = DEC(10)
pretrain(train_loader, test_loader, 200)
# dec.train(train_loader, test_loader, 100)

In [ ]:
model =DEC_AE(10,10).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/MINI_MODELS/Final_model_conv.pt', map_location=device)['model_state_dict'], strict=False)

model.eval()